# Introduction 

This notebook replicates a simple chat template with continuous chat. The model understands and remembers the context according to its capacity. We use the *chat template* functionality of the tokenizer.

This is a Phi 1.5 model fine-tuned to Chat Alpaca dataset (https://huggingface.co/datasets/flpelerin/ChatAlpaca-10k). Find the fine-tuning notebook in the `assistant_sft` directory.

**NOTE: The notebook uses a customized streamer for text streaming.**

In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    logging,
)

from streaming_utils import TextStreamer

In [2]:
model = AutoModelForCausalLM.from_pretrained(
    '../assistant_sft/outputs/phi_1_5_chat_alpaca/logs/checkpoint-1187/',
    load_in_4bit=True
)
tokenizer = AutoTokenizer.from_pretrained(
    '../assistant_sft/outputs/phi_1_5_chat_alpaca/logs/checkpoint-1187/'
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
streamer = TextStreamer(
    tokenizer,
    skip_prompt=True,
    skip_special_tokens=True,
    truncate_before_pattern=['\[\/', 'Goodbye'],
    truncate=True
)

In [4]:
eos_string = tokenizer.eos_token
history = None

In [5]:
chat = [
       {"from": "human", "value": 'Hello'}
    ]

tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

'<|endoftext|>[INST] Hello [/INST]'

In [6]:
while True:
    question = input("Question: ")

    chat = [
       {"from": "human", "value": question}
    ]

    template = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

    prompt = history + ' ' + template if history is not None else template

    # print(prompt)
    
    prompt_tokenized = tokenizer(
        prompt, 
        return_tensors='pt', 
        padding=True, 
        truncation=True,
        return_attention_mask=True
    ).to('cuda')['input_ids']
    
    output_tokenized = model.generate(
        input_ids=prompt_tokenized,
        max_length=len(prompt_tokenized[0])+400,
        temperature=0.7,
        top_k=40,
        top_p=0.95,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
        streamer=streamer
    )
    answer = tokenizer.decode(token_ids=output_tokenized[0][len(prompt_tokenized[0]):]).strip()

    # Puny guardrails.
    if eos_string in answer:
        answer = answer.split(eos_string)[0].strip()
    if '[/' in answer:
        answer = answer.split('[/')[0].strip()

    history = ' '.join([prompt, answer])
    # print(f"ANSWER: {answer}\n")
    # print(f"HISTORY: {history}\n")
    print('#' * 50)

Question:  Hi. How are you?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/home/sovitdl/miniconda3/envs/experiments/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


I am doing great! I'm currently studying for my degree in computer science and working on a project to develop an AI system that can recognize handwritten text. It's challenging but rewarding work so far, as the results have been promising. Thanks for asking! What about yourself? 
##################################################


Question:  I am doing deep learning.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

No worries. Just wanted to check if you've ever heard of any other ways that deep learning can be applied in our daily lives besides gaming or virtual assistants.

Is there anything you'd like to know more about? 

Looking forward to hearing back from you soon. 

Best Regards, 
AI language model. 

(Note: The AI language model does not provide a direct answer to the initial prompt, but offers assistance by suggesting other potential uses of deep learning.) 

Great to hear from you again!

Regards,
The AI language model.


##################################################


Question:  Let's talk about mathematics now.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sure thing! Mathematics is a fascinating subject because it allows us to understand patterns, solve problems, and make predictions about the world around us. Here are a few key ideas to keep in mind:

1. Numbers: At its most basic level, math involves manipulating numbers to find solutions. We use arithmetic, algebra, geometry, and calculus among others to study the properties of numbers and relationships between them.

2. Patterns: Math is full of patterns - whether it's finding repeating sequences, identifying trends in data sets, or recognizing geometric shapes. Understanding patterns requires logical thinking and spatial reasoning 

KeyboardInterrupt: 